In [1]:
# this script takes satellite data from catalog, 
# parses to place on regularly gridded array, and interpolates nan across desired gaps in space and time

import numpy as np
import xarray as xr 
import time 
from intake import open_catalog
from matplotlib.colors import LogNorm
import matplotlib
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy import integrate
from datetime import datetime
from tqdm.notebook import tqdm
import pickle 
from altimetry_tools import Haversine, nan_helper, interp_nans, parse_grid_tracks, smooth_tracks, coarsen, specsharp, spectra_slopes
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

# -- main PARAMETERS --- 
this_sat = 's3a'              # mission to consider 
hor_grid_spacing = 10        # km, grid to which to interpolate tracks
interp_cutoff = 5            # number of acceptable nan gaps in grid cell units across which to interpolate
coarsening_factor0 = 7       # (*hor_grid_spacing = coarsened grid size) this factor should be multiplied by hor_grid_spacing 
nyquist_wavelength = np.pi   # factor relative to coarsening factor (what scales do we want to resolve...pi times the grid scale)
f_type = 'sharp'             # filter type (gaussian or sharp)
# -----------------------
# -- coastline file 
x4 = xr.open_dataset('/home/jovyan/along-track-altimetry-analysis/misc/coastlines_global.nc')  
# -- deformation radius 
# Chelton et al. 1998 'global variability of the first baroclinic rossby radius of deformation'
c98 = xr.open_dataset('/home/jovyan/along-track-altimetry-analysis/misc/global_deformation_radius_chelton_1998.nc')  
c98 = c98['values'].data
# (http://www-po.coas.oregonstate.edu/research/po/research/rossby_radius/index.html#anchor2)
# -- catalog of available satellites 
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/altimetry.yaml")# load individual satellite 
# ['al', 'alg', 'c2', 'e1', 'e1g', 'e2', 'en', 'enn', 'g2', 'h2', 'j1', 'j1g', 'j1n', 'j2', 'j2g', 'j2n', 'j3', 's3a', 's3b', 'tp', 'tpn']

# -- lat/lon boundaries (permits subsetting to speed processing time)
lon_w = 0
lon_e = 360
lat_s = -65
lat_n = 65

In [2]:
# ------------------------------------------------------------------------------------------------
# RUN ONLY if parsed tracks file doesn't exist (should have a parsed file for each satellite)
# ------------------------------------------------------------------------------------------------
# load individual sat (your choice) and convert to dataframe from dataset 
ds2_0 = cat[this_sat].to_dask()
%time ds2 = ds2_0[['latitude', 'longitude', 'sla_unfiltered', 'sla_filtered', 'track', 'cycle', 'mdt']].reset_coords().astype('f4').load()
%time df2 = ds2.to_dataframe()
df2_s = df2[(df2['longitude'] > lon_w) & (df2['longitude'] < lon_e) & (df2['latitude'] > lat_s) & (df2['latitude'] < lat_n)]
in_tracks2 = np.unique(df2_s['track'])
print('tracks in domain')
print(in_tracks2)

test = df2_s[df2_s['track'] == 11]  # 9 is standard
p = test.index
ts = (p - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
test2 = np.where(np.diff(ts) > 24*60*60)[0]
repeat_time = p[test2[3]] - p[test2[2]]
print('repeat time = ' + str(repeat_time))

test0 = df2_s[df2_s['track'] == in_tracks2[10]]  # 11
in_cycles0 = np.unique(test0['cycle'])
test1 = test0[test0['cycle'] == in_cycles0[0]]  # 20
ii = 5
d = Haversine(test1['latitude'][ii], test1['longitude'][ii], test1['latitude'][ii-1], test1['longitude'][ii-1])
print('nominal grid spacing = ' + str(d) + 'km')

CPU times: user 5.83 s, sys: 2.79 s, total: 8.62 s
Wall time: 3.85 s
CPU times: user 214 ms, sys: 292 ms, total: 506 ms
Wall time: 504 ms
tracks in domain
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.
  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.
  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.
  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.
  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.
  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.  98.
  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112.
 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126.
 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140.
 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153. 154.
 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166. 167

In [3]:
ds2_0

<xarray.Dataset>
Dimensions:         (time: 48452566)
Coordinates:
    latitude        (time) float64 dask.array<chunksize=(24226283,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(24226283,), meta=np.ndarray>
  * time            (time) datetime64[ns] 2016-06-15T00:25:27.573921024 ... 2...
Data variables:
    cycle           (time) int16 dask.array<chunksize=(48452566,), meta=np.ndarray>
    dac             (time) float64 dask.array<chunksize=(24226283,), meta=np.ndarray>
    lwe             (time) float64 dask.array<chunksize=(24226283,), meta=np.ndarray>
    mdt             (time) float64 dask.array<chunksize=(24226283,), meta=np.ndarray>
    ocean_tide      (time) float64 dask.array<chunksize=(24226283,), meta=np.ndarray>
    sla_filtered    (time) float64 dask.array<chunksize=(24226283,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(24226283,), meta=np.ndarray>
    track           (time) int16 dask.array<chunksize=(48452566,), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.6
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Swath
    comment:                   Sea surface height measured by altimeters refe...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    creator_email:             servicedesk.cmems@mercator-ocean.eu
    creator_name:              CMEMS - Sea Level Thematic Assembly Center
    creator_url:               http://marine.copernicus.eu
    history:                   2019-02-20T17:35:26Z: Creation
    institution:               CLS, CNES, EUMETSAT
    keywords:                  Oceans > Ocean Topography > Sea Surface Height
    keywords_vocabulary:       NetCDF COARDS Climate and Forecast Standard Names
    license:                   http://marine.copernicus.eu/web/27-service-com...
    platform:                  Sentinel-3A
    processing_level:          L3
    product_version:           2019
    project:                   EUMETSAT Sentinel-3 L2P/L3 Marine Altimetry Se...
    references:                http://marine.copernicus.eu
    software_version:          6.2_DUACS_DT2018_baseline
    source:                    Sentinel-3A measurements
    ssalto_duacs_comment:      The reference mission used for the altimeter i...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   SSALTO/DUACS Delayed-Time Level-3 sea surface ...
    title:                     DT Sentinel-3A Global Ocean Along track SSALTO...

In [3]:
# ------------------------------------------------------------------------------------------------
# RUN ONLY if parsed tracks file doesn't exist (should have a parsed file for each satellite)
# ------------------------------------------------------------------------------------------------

# -- PARSE_grid_tracks (interpolate nans and group by track) (sla dimensions = [track_number][cycle, along_track_grid])
f_v_uf = 1  # if 1, use filtered product from aviso 
%time lon_t, lat_t, track_t, adt, sla, dist, lon_record, lat_record, time_record, track_record \
    = parse_grid_tracks(in_tracks2, df2_s, hor_grid_spacing, interp_cutoff, f_v_uf)

del sla # make more memory space 
# ---------------   
# -- export to pickle so that we don't have to run/parse everytime 
# ---------------
save_p = 1
if save_p > 0:
    outputs = {'lon_t': lon_t, 'lat_t': lat_t, 'track_t': track_t, 'adt': adt, \
               'dist': dist, 'lon_record': lon_record, 'lat_record': lat_record, \
               'time_record': time_record, 'track_record': track_record, \
               'repeat_time': repeat_time, 'interp_cutoff': interp_cutoff}
    pickle.dump(outputs, open(this_sat + '/' + this_sat + '_parsed_tracks_adt.p', 'wb'))
# ------------------------------------------------------------------------------------------------


CPU times: user 24min 48s, sys: 11 s, total: 24min 59s
Wall time: 24min 59s


In [2]:
# ------------------------------------------------------------------------------------------------
# RUN AFTER parse_grid_tracks is completed  (should have a parsed file for each satellite)
# now interpolate in time (input should be output file from above cell)
# ------------------------------------------------------------------------------------------------
load_sat = pickle.load(open(this_sat + '/' + this_sat +'_parsed_tracks_adt.p', 'rb'))
adt = load_sat['adt']
# -- interpolate in time 
interp_cutoff_t = 5  # time step to interpolate across (number samples, not a unit of time)
adt_time = []
sla_time = []
# - loop over tracks 
for i in tqdm(range(len(adt))):
    # - loop over along-track gridpoints 
    # this_interp_t_sla = np.nan * np.ones(np.shape(sla[i]))
    this_interp_t_adt = np.nan * np.ones(np.shape(adt[i]))
    for j in range(np.shape(adt[i])[1]): 
        # this_data_1 = sla[i][:, j]
        this_data_2 = adt[i][:, j]
        # if np.sum(np.isnan(this_data_1)) > 0:
        #     nans, x = nan_helper(this_data_1)
        #     this_interp_t_sla[:, j] = interp_nans(this_data_1, nans, x, interp_cutoff_t)
        # else:
        #     this_interp_t_sla[:, j] = this_data_1.copy()
        if np.sum(np.isnan(this_data_2)) > 0:
            nans, x = nan_helper(this_data_2)
            this_interp_t_adt[:, j] = interp_nans(this_data_2, nans, x, interp_cutoff_t)
        else:
            this_interp_t_adt[:, j] = this_data_2.copy()
    # sla_time.append(this_interp_t_sla)
    adt_time.append(this_interp_t_adt)
    
save_p = 1
if save_p > 0:
    outputs = {'adt': adt_time}
    pickle.dump(outputs, open(this_sat + '/' + this_sat + '_parsed_tracks_adt_interp_time_only.p', 'wb'))